*created by Laura Brin and Jaskaran Singh for CMPT3830: Machine Learning Work Integrated Project at Norquest College*

# Board Game Ratings

### Board Game Insights
The board game industry is a growing industry. Valued at 13.75 billion USD in 2021, this sector is expected to increase in value to 30.93 billion by 2028 (Globenewswire). There is an increasing trend among board game enthusiasts towards independent developers with word of mouth and kickstarter being the main way of learning about new games (PrintNinja). The ideal user of this application will be indie developers looking to improve their decisions around developing new games for the market. It could also be used by hobby stores to visualize trends to help make purchasing decisions. <br>

The product backlog for this project is a bit ambitious. We would like to provide multiple options for potential game developers or creators to interact with the models. The main output will be an estimation of rating based on provided information about a game the user is developing or looking to acquire. By experimenting with different game details the user can evaluate if there would be value in making changes to their game. We would also like to develop an interactive way to view time-series trends using ipywidgets. <br>

Additional features that would be great to introduce but will be outside of the time scope available include using keywords to return likelihood of kickstarter success and/or potential game rating and inputting game information and receiving a ranking of best months/quarters to launch the game in kickstarter. <br>

load libraries

In [ ]:
import sqlite3 
import pandas as pd
from zipfile import ZipFile

importing datasets

In [ ]:
#importing datasets for board games

zf = ZipFile('database.zip')
#extracted data is saved in the same directory as notebook
zf.extractall() 
zf.close()

conn = sqlite3.connect("database.sqlite")
cur = conn.cursor()
df_games= pd.read_sql_query("SELECT* FROM BoardGames",conn)
#conn.close()


#importing datasets for Kickstarter

zf = ZipFile('ks-projects-201801.zip')
#extracted data is saved in the same directory as notebook
zf.extractall() 
zf.close()

df_ks=pd.read_csv("ks-projects-201801.csv")

 ### Games Dataset-Laura

In [ ]:
#separating df_games columns to useable sections using game.id as key
details=["game.id","game.type","details.description","details.image","details.maxplayers","details.maxplaytime","details.minage",
"details.minplayers","details.minplaytime","details.name","details.playingtime","details.thumbnail","details.yearpublished"]
attributes=["game.id","attributes.boardgameartist","attributes.boardgamecategory","attributes.boardgamecompilation","attributes.boardgamedesigner","attributes.boardgameexpansion",
"attributes.boardgamefamily","attributes.boardgameimplementation","attributes.boardgameintegration","attributes.boardgamemechanic","attributes.boardgamepublisher","attributes.total","attributes.t.links.concat.2...."]
stats=["game.id","stats.average","stats.averageweight","stats.bayesaverage","stats.family.abstracts.bayesaverage","stats.family.cgs.bayesaverage","stats.family.cgs.pos",
"stats.family.childrensgames.bayesaverage","stats.family.childrensgames.pos","stats.family.familygames.bayesaverage","stats.family.familygames.pos",
"stats.family.partygames.bayesaverage","stats.family.partygames.pos","stats.family.strategygames.bayesaverage","stats.family.strategygames.pos","stats.family.thematic.bayesaverage","stats.family.thematic.pos",
"stats.family.wargames.bayesaverage","stats.family.wargames.pos","stats.median","stats.numcomments","stats.numweights","stats.owned","stats.stddev","stats.subtype.boardgame.bayesaverage",
"stats.subtype.boardgame.pos","stats.trading","stats.usersrated","stats.wanting","stats.wishing","stats.family.amiga.bayesaverage","stats.family.amiga.pos","stats.family.arcade.bayesaverage","stats.family.arcade.pos","stats.family.atarist.bayesaverage","stats.family.atarist.pos",
"stats.family.commodore64.bayesaverage","stats.family.commodore64.pos","stats.subtype.rpgitem.bayesaverage","stats.subtype.rpgitem.pos","stats.subtype.videogame.bayesaverage","stats.subtype.videogame.pos"]
polls=["game.id","polls.language_dependence","polls.suggested_numplayers.1","polls.suggested_numplayers.10","polls.suggested_numplayers.2","polls.suggested_numplayers.3","polls.suggested_numplayers.4",
"polls.suggested_numplayers.5","polls.suggested_numplayers.6","polls.suggested_numplayers.7","polls.suggested_numplayers.8","polls.suggested_numplayers.9","polls.suggested_numplayers.Over","polls.suggested_playerage"]

In [ ]:
#creating smaller dataframes to work with
df_games_details=df_games[details]
df_games_attributes=df_games[attributes]
df_games_stats=df_games[stats]
df_games_polls=df_games[polls]

In [ ]:
df_games_details.head(10)

In [ ]:
df_games_details.dtypes

###  Kickstarter dataset-Jaskaran

In [ ]:
df_ks.head(10)

### References

Board Game Industry Statistics (n.d.) PrintNinja. (https://printninja.com/board-game-industry-statistics/)

SkyQuest Technology (2022, July 19) Board Games Market to Attain Value of $30.93 Billion By 2028 Thanks to Increased Popularity of Online Gaming and Entry of New OTT platforms In Board Gaming. Globe News Wire. (https://www.globenewswire.com/news-release/2022/07/19/2482068/0/en/Board-Games-Market-to-Attain-Value-of-30-93-Billion-By-2028-Thanks-to-Increased-Popularity-of-Online-Gaming-and-Entry-of-New-OTT-platforms-In-Board-Gaming.html)

https://www.gamedesignworkshop.com/understanding-the-tabletop-game-industry
